## 투빅스 18기 Week5 SVM 과제 - 18기 이다인

# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [4]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

* multiclass SVM에서 One vs One SVM을 구현하였고, class로 구현했습니다.
- 이번 과제는 제가 직접 multiclass svm을 구현하는 것이었는데, 과제를 진행하면서 제가 아직 svm을 구현할 수 있는 능력이 안되는 것 같다고 느꼈습니다... 그래서 우수과제에 있는 코드를 분석해보는 식으로 과제를 진행하였습니다. 죄송합니다

**OVO (One vs. One)**

In [6]:
from itertools import combinations

In [12]:
# iris data만이 아니라 다른 모든 데이터에서도 적용이 가능해야하기 때문에, label의 class수를 직접 입력받고,
# 분류기도 class의 조합의 수만큼 만들어냈음
class OnevsOne_SVM:
    def __init__(self, n_classes, kernel, C, gamma):
        self.n_classes = n_classes
        #label의 class수를 입력받음
        self.clfs = [{'class' : None,'clf' : SVC(kernel = kernel, C = C, gamma = gamma)} for _ in range(int(n_classes * (n_classes-1) / 2))]
        #kernel, C, gamma값을 입력값으로 받아 설정해줌
        #분류모델의 개수는 클래스가 n개일 때 nC2=n(n−1)/2이므로 n_classes의 조합의 수만큼 분류기를 만들어줌
        self.classes = None
        self.combi = []
        
    def fit(self, X_train, y_train):
        self.classes = y_train.unique()
        #['virginica', 'versicolor', 'setosa']배열로 classes 설정
        i = 0
        # combinations(self.classes, 2) = nC2
        for c in combinations(self.classes, 2):
            idx = (y_train == c[0]) | (y_train == c[1])
            # classes의 조합 별로 idx를 나눔
            self.clfs[i]['clf'].fit(X_train[idx], y_train[idx])
            # 조합별로 svm을 훈련시킴
            self.clfs[i]['class'] = c
            self.combi.append(c)
            i += 1
            
    def predict(self, X_test):
        preds_df = pd.DataFrame([svm['clf'].predict(X_test) for svm in self.clfs]).T 
        # 각 조합 별로 예측값을 구해줌 (.T를 통해서 전치행렬로 바꿔줌)
        decisions = pd.DataFrame([svm['clf'].decision_function(X_test) for svm in self.clfs]).T 
        # (결정경계로부터 거리의 합을 구해주기 위함) decision_fuction을 이용하여 
        # 각 클래스 별 거리를 구해주고 전치행렬로 바꾸어줌
        decisions.columns = self.combi
        # 각 클래스 별 거리의 column을 위에서 만들어놓은 self.combi로 바꾸어줌
        
        final_pred = []
        for i in range(len(preds_df)):
            
            # case1) 위의 예측들 중 가장 많이 예측 한 클래스가 있을 경우 해당 클래스를 리스트에 저장해준다
            if preds_df.iloc[i].value_counts().iloc[0] > preds_df.iloc[i].value_counts().iloc[1]:
                label = (preds_df.iloc[i].value_counts() / len(preds_df.iloc[i])).index[0]
                final_pred.append(label)
            
            # case1으로 분류가 안되는 경우
            else:
                decision_for_row = {key : 0 for key in classes}
                for c, d in zip(decisions.iloc[i].index, decisions.iloc[i]):
                    # case2) 예측값 중 겹치는 클래스가 존재하는 경우 결정경계로부터 거리의 합이 큰 클래스를 선택한다
                    if d > 0:
                        decision_for_row[c[0]] += d
                    # case3) 예측값의 모든 클래스가 다를 경우 마찬가리로 결정경계로부터 거리의 합이 큰 클래스를 선택한다
                    else:
                        decision_for_row[c[1]] -= d
                label = sorted(decision_for_row.items(), key = lambda x : x[1], reverse = True)[0][0]
                final_pred.append(label)
        return final_pred
    

In [13]:
#위에서 만들어높은 class를 적용시킴
clf = OnevsOne_SVM(n_classes = 3, kernel = 'rbf', C = 5, gamma = 5)
#위에서 만들어놓은 fit, predict함수를 적용시킨다
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [14]:
accuracy_score(y_test, pred)

0.8666666666666667